In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import seaborn as sns
import timedelta

plt.rcParams['font.family'] = 'Malgun Gothic'  # or 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import numpy as np
from scipy.spatial import cKDTree
pd.set_option('display.max_columns', None)

## 기상 관측 지점 데이터 불러오기

In [5]:
loc_df = pd.read_csv("./data/meta-loc-edited.csv", encoding='cp949')
loc_gdf = gpd.GeoDataFrame(
    loc_df, geometry=gpd.points_from_xy(loc_df.경도, loc_df.위도), crs="EPSG:4326"
)
loc_gdf.지점 = loc_gdf.지점.apply(str)

## 기상 관측 정보에 있는 지점들로 전처리 

In [ ]:
# 기상 관측 정보 불러오기 (기상데이터는 미업로드)
weather = gpd.read_file("./data/whetherDataEH.csv", encoding='cp949')

In [ ]:
weather

,지점,지점명,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),실효습도
0,12,안면도(감),2022-01-01 00:00,-4.5,34.0,2.6,0.0,59.0,17.700000000000003
1,12,안면도(감),2022-01-01 01:00,-4.6,37.8,2.4,0.0,58.0,29.790000000000003
2,12,안면도(감),2022-01-01 02:00,-5.0,62.6,2.7,0.0,65.0,31.680000000000003
3,12,안면도(감),2022-01-01 03:00,-5.0,68.4,2.9,0.0,54.0,29.850000000000005
4,12,안면도(감),2022-01-01 04:00,-4.9,65.6,2.7,0.0,51.0,26.640000000000004
...,...,...,...,...,...,...,...,...,...
16418967,996,화동,2024-12-31 19:00,-0.5,268.2,3.1,0.0,38.0,18.960000000000004
16418968,996,화동,2024-12-31 20:00,-0.8,266.5,3.8,0.0,38.0,19.380000000000003
16418969,996,화동,2024-12-31 21:00,-1.6,262.5,5.2,0.0,46.0,21.78
16418970,996,화동,2024-12-31 22:00,-2.4,263.6,3.9,0.0,55.0,26.16


In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16418972 entries, 0 to 16418971
Data columns (total 9 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   지점       object
 1   지점명      object
 2   일시       object
 3   기온(°C)   object
 4   풍향(deg)  object
 5   풍속(m/s)  object
 6   강수량(mm)  object
 7   습도(%)    object
 8   실효습도     object
dtypes: object(9)
memory usage: 1.1+ GB


In [ ]:
# 기상 관측 정보에 있는 지점들 찾기
loc_gdf_needed = loc_gdf[loc_gdf.지점.isin(weather.지점.unique())]
loc_gdf_needed

,지점,시작일,종료일,지점명,위도,경도,geometry
0,12,2007-11-30,NaN,안면도(감),36.5333,126.3167,POINT (126.3167 36.5333)
1,96,2020-10-23,NaN,독도,37.2395,131.8698,POINT (131.8698 37.2395)
2,116,2023-10-21,NaN,관악(레),37.4453,126.9640,POINT (126.964 37.4453)
4,160,2020-10-23,NaN,부산(레),35.1188,129.0000,POINT (129 35.1188)
5,175,2024-10-15,NaN,진도(레),34.4721,126.3238,POINT (126.3238 34.4721)
...,...,...,...,...,...,...,...
567,992,2021-11-10,NaN,하빈,35.9062,128.4464,POINT (128.4464 35.9062)
568,993,2024-04-06,NaN,제주금악,33.3423,126.3099,POINT (126.3099 33.3423)
569,994,2022-05-02,NaN,심포,35.8545,126.6421,POINT (126.6421 35.8545)
570,995,2023-12-19,NaN,오천,35.9286,129.3822,POINT (129.3822 35.9286)


In [ ]:
loc_gdf

,지점,시작일,종료일,지점명,위도,경도,geometry
0,12,2007-11-30,NaN,안면도(감),36.5333,126.3167,POINT (126.3167 36.5333)
1,96,2020-10-23,NaN,독도,37.2395,131.8698,POINT (131.8698 37.2395)
2,116,2023-10-21,NaN,관악(레),37.4453,126.9640,POINT (126.964 37.4453)
3,144,1993-03-16,2011-12-09,군산(레),36.0123,126.7834,POINT (126.7834 36.0123)
4,160,2020-10-23,NaN,부산(레),35.1188,129.0000,POINT (129 35.1188)
...,...,...,...,...,...,...,...
567,992,2021-11-10,NaN,하빈,35.9062,128.4464,POINT (128.4464 35.9062)
568,993,2024-04-06,NaN,제주금악,33.3423,126.3099,POINT (126.3099 33.3423)
569,994,2022-05-02,NaN,심포,35.8545,126.6421,POINT (126.6421 35.8545)
570,995,2023-12-19,NaN,오천,35.9286,129.3822,POINT (129.3822 35.9286)


In [ ]:
loc_array = loc_gdf['지점명'].unique()

# 데이터 열어보기

In [ ]:
fire_data = gpd.read_file("./fire_clean.gpkg") # 기상지점명 매핑된 산불 데이터

In [ ]:
fire_data


,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계,발생장소_읍면동,발생장소_시도_시군구,발생장소_시도_시군구_읍면동,CTPRVN_CD,SIG_CD,EMD_CD,매핑수준,매핑방법,발생일시,진화일시,duration,index_right,지점명,distance,geometry
0,전북,전북,남원,산동,부절,기,작업장실화,산업현장실화,0.31,산동,전북 남원,전북 남원 산동,45,45190,45190410,완전 매칭,"시도,시군구,읍면동",2024-09-29 15:41:00,2024-09-30 16:30:00,24.816667,63,번암,0.075951,POINT (127.48083 35.50199)
1,경남,경남,밀양,None,내이,기,기타(직접입력),성묘객실화(벌집소각),0.10,내이,경남 밀양,경남 밀양 내이,48,48270,48270102,완전 매칭,"시도,시군구,읍면동",2024-09-10 15:55:00,2024-09-10 18:00:00,2.083333,522,생림,0.146043,POINT (128.74364 35.497)
2,충남,충남,부여,규암,수목,기,기타(직접입력),원인미상,0.03,규암,충남 부여,충남 부여 규암,44,44760,44760310,완전 매칭,"시도,시군구,읍면동",2024-09-10 14:35:00,2024-09-10 17:52:00,3.283333,296,정산,0.137384,POINT (126.86571 36.28309)
3,경북,경북,상주,화동,신촌,기,기타(직접입력),조사중,1.00,화동,경북 상주,경북 상주 화동,47,47250,47250400,완전 매칭,"시도,시군구,읍면동",2024-09-10 14:24:00,2024-09-10 22:00:00,7.600000,435,화서,0.051610,POINT (127.97244 36.398)
4,경북,경북,안동,녹전,매정,None,기타(직접입력),농산폐기물소각,0.05,녹전,경북 안동,경북 안동 녹전,47,47170,47170430,완전 매칭,"시도,시군구,읍면동",2024-09-05 13:51:00,2024-09-05 16:10:00,2.316667,438,이산,0.145530,POINT (128.76731 36.75129)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,대구,대구,달성,옥포,반송,기,기타(직접입력),화목난로비화,0.08,옥포,대구 달성,대구 달성 옥포,27,27710,27710262,완전 매칭,"시도,시군구,읍면동",2022-01-03 12:36:00,2022-01-03 15:35:00,2.983333,566,옥포,0.056413,POINT (128.48786 35.76543)
1512,강원,강원,인제,남,수산,기,기타(직접입력),화목보일러재투기,0.80,남,강원 인제,강원 인제 남,51,51810,51810310,완전 매칭,"시도,시군구,읍면동",2022-01-03 12:02:00,2022-01-03 14:56:00,2.900000,193,신남,0.048761,POINT (128.10488 37.9979)
1513,전남,전남,보성,조성,귀산,담,담뱃불실화,담뱃불실화,0.10,조성,전남 보성,전남 보성 조성,46,46780,46780370,완전 매칭,"시도,시군구,읍면동",2022-01-02 14:29:00,2022-01-02 16:50:00,2.350000,349,벌교,0.126323,POINT (127.24196 34.80107)
1514,경북,대구,군위,부계,신화,None,주택화재비화,축사전기누전,0.20,부계,대구 군위,대구 군위 부계,27,27720,27720330,완전 매칭,"시도,시군구,읍면동",2022-01-02 08:18:00,2022-01-02 12:50:00,4.533333,454,팔공산,0.062770,POINT (128.6619 36.06617)


In [ ]:
fire_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   발생장소_관서          1516 non-null   object        
 1   발생장소_시도          1516 non-null   object        
 2   발생장소_시군구         1508 non-null   object        
 3   발생장소_읍면          1234 non-null   object        
 4   발생장소_동리          1516 non-null   object        
 5   발생원인_구분          1300 non-null   object        
 6   발생원인_세부원인        1516 non-null   object        
 7   발생원인_기타          1516 non-null   object        
 8   피해면적_합계          1516 non-null   float64       
 9   발생장소_읍면동         1516 non-null   object        
 10  발생장소_시도_시군구      1508 non-null   object        
 11  발생장소_시도_시군구_읍면동  1508 non-null   object        
 12  CTPRVN_CD        1516 non-null   object        
 13  SIG_CD           1516 non-null   object        
 14  EMD_CD           1516 non-null  

In [ ]:
# 기상 정보 기준 지점명이 매핑된 삼림 데이터 (업로드되지않음)
weather_loc_fist100 = gpd.read_file("./data/weather_Loc&frst100.gpkg")

In [ ]:
weather_loc_fist100

,index,FIFTH_FRTP,DMCLS_CD,AGCLS_CD,DNST_CD,frtp_CD,index_right,지점명,distance,geometry
0,0,H,2,4,B,활엽수림,118,관악,0.029679,"MULTIPOLYGON (((126.92156 37.44277, 126.92144 ..."
1,1,H,2,4,B,활엽수림,74,기상청,0.005298,"MULTIPOLYGON (((126.91631 37.48976, 126.91672 ..."
2,2,H,2,4,B,활엽수림,74,기상청,0.005523,"MULTIPOLYGON (((126.92213 37.49274, 126.92385 ..."
3,3,H,2,5,B,활엽수림,81,금천,0.019143,"MULTIPOLYGON (((126.91117 37.45484, 126.91303 ..."
4,4,H,2,5,B,활엽수림,81,금천,0.026817,"MULTIPOLYGON (((126.92227 37.45497, 126.92483 ..."
...,...,...,...,...,...,...,...,...,...,...
262843,501718,Q,2,4,C,활엽수림,410,대덕,0.125225,"MULTIPOLYGON (((127.86824 35.99774, 127.87158 ..."
262844,501719,Q,2,5,B,활엽수림,410,대덕,0.127180,"MULTIPOLYGON (((127.8648 35.99199, 127.8671 35..."
262845,501720,Q,2,5,C,활엽수림,410,대덕,0.119488,"MULTIPOLYGON (((127.87263 35.98754, 127.87137 ..."
262846,501721,Q,2,5,C,활엽수림,410,대덕,0.121460,"MULTIPOLYGON (((127.875 35.99708, 127.875 35.9..."


In [ ]:
weather_loc_fist100.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 262848 entries, 0 to 262847
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   index        262848 non-null  int64   
 1   FIFTH_FRTP   262848 non-null  object  
 2   DMCLS_CD     262848 non-null  int32   
 3   AGCLS_CD     262848 non-null  int32   
 4   DNST_CD      262848 non-null  object  
 5   frtp_CD      262848 non-null  object  
 6   index_right  262848 non-null  int64   
 7   지점명          262848 non-null  object  
 8   distance     262848 non-null  float64 
 9   geometry     262848 non-null  geometry
dtypes: float64(1), geometry(1), int32(2), int64(2), object(4)
memory usage: 18.0+ MB


In [ ]:
# 삼림 데이터에 있는 지점 개수
wt_loc_array = weather_loc_fist100['지점명'].unique()

In [ ]:
# 기상 정보 데이터와 삼림 데이터의 지점명 차이 찾기
diff1 = np.setdiff1d(loc_array, wt_loc_array) # 기상 - 삼림
diff1

array(['가거도', '거문도', '교동', '군산(레)', '금악', '김화', '남항', '대연평', '덕적북리',
       '덕적지도', '도라산', '독도', '마라도', '마현', '말도', '목덕도', '백령(레)', '백령면',
       '백학', '보길도', '볼음도', '북격렬비도', '북악산', '북한산', '북항', '상서', '색달',
       '서거차도', '선유도', '세종연기', '소청도', '송도', '순창', '스키점프', '승봉도', '신서',
       '양도', '양지', '어청도', '여서도', '영등포', '영월주천', '영흥도', '옹도', '외연도', '외촌',
       '용산', '우도', '웅상', '위도', '임남', '자월도', '정선북', '죽정', '중면', '진도읍',
       '철원장흥', '청산도', '초도', '추자도', '판문점', '평도', '풍도', '하태도', '향로봉', '현내',
       '호도', '홍도'], dtype=object)

In [ ]:
diff2 = np.setdiff1d(wt_loc_array, loc_array) # 삼림 - 기상은 없음
diff2 # 삼림 데이터가 기상 데이터의 지점명을 따름을 확인

array([], dtype=object)

# 기상 정보에 산불 정보 라벨링

In [ ]:
# 일시 컬럼 형식을 datetime으로 변환
weather['일시'] = pd.to_datetime(weather['일시'], errors='coerce')
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16418972 entries, 0 to 16418971
Data columns (total 9 columns):
 #   Column   Dtype         
---  ------   -----         
 0   지점       object        
 1   지점명      object        
 2   일시       datetime64[ns]
 3   기온(°C)   object        
 4   풍향(deg)  object        
 5   풍속(m/s)  object        
 6   강수량(mm)  object        
 7   습도(%)    object        
 8   실효습도     object        
dtypes: datetime64[ns](1), object(8)
memory usage: 1.1+ GB


In [ ]:
#좌표계변환
fire_data = fire_data.to_crs(epsg=5179)
weather_loc_fist100 = weather_loc_fist100.to_crs(epsg=5179)

### 삼림 데이터의 위치&매핑된 기상 정보 지점을 산불 데이터의 위치와 매핑한다

In [ ]:
#넓이 변환
fire_data['피해면적_m^2'] = fire_data['피해면적_합계'] * 10000
fire_data['반지름'] = np.sqrt(fire_data['피해면적_m^2'] / np.pi)
fire_data['반지름'] = fire_data['반지름'].fillna(0).clip(lower=0)

In [ ]:
fire_data['rad_buffer'] = fire_data.geometry.buffer(fire_data['반지름'])
fire_data_buffers_gdf = fire_data.set_geometry('rad_buffer')

In [ ]:
# 산불 버퍼와 삼림 데이터 spatial join
sjoin_result = gpd.sjoin(fire_data_buffers_gdf[['발생일시', '진화일시', 'rad_buffer']], weather_loc_fist100[['지점명', 'geometry']], how='left', predicate='intersects')

In [ ]:
overlapping_branches = sjoin_result.groupby(sjoin_result.index)['지점명'].agg(lambda x: list(x.dropna().unique()))
fire_data['overlapping_forest_branches'] = fire_data.index.map(overlapping_branches)

In [ ]:
sjoin_result[sjoin_result['지점명'].notnull()]

,발생일시,진화일시,rad_buffer,index_right,지점명
2,2024-09-10 14:35:00,2024-09-10 17:52:00,"POLYGON ((943047.385 1809719.634, 943047.338 1...",62277.0,정산
4,2024-09-05 13:51:00,2024-09-05 16:10:00,"POLYGON ((1113141.471 1862216.357, 1113141.411...",158913.0,이산
4,2024-09-05 13:51:00,2024-09-05 16:10:00,"POLYGON ((1113141.471 1862216.357, 1113141.411...",158900.0,이산
6,2024-08-31 14:54:00,2024-08-31 18:00:00,"POLYGON ((1047130.865 1695503.4, 1047130.743 1...",176106.0,단성
7,2024-08-25 13:27:00,2024-08-25 17:00:00,"POLYGON ((1091014.012 1816279.894, 1091013.951...",129866.0,소보
...,...,...,...,...,...
1502,2022-01-06 13:47:00,2022-01-06 15:40:00,"POLYGON ((974830.533 1865658.73, 974830.428 18...",78026.0,세종전의
1508,2022-01-04 12:25:00,2022-01-04 15:50:00,"POLYGON ((1111077.965 1887048.394, 1111077.772...",164392.0,부석
1509,2022-01-04 10:36:00,2022-01-04 13:30:00,"POLYGON ((1029947.478 1987478.305, 1029947.356...",227971.0,춘천신북
1510,2022-01-04 02:34:00,2022-01-04 03:30:00,"POLYGON ((1169511.493 1827847.449, 1169511.433...",149980.0,영덕읍


기상 정보 데이터에 산불 칼럼을 추가해 매핑

In [ ]:
# 칼럼 초기화 (산불이 없는 경우 0)
weather['산불'] = 0

In [ ]:
valid_fire_data = fire_data[fire_data['overlapping_forest_branches'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)].itertuples()
num_valid_fire_data = fire_data['overlapping_forest_branches'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False).sum()

In [ ]:
for index, fire_event in enumerate(valid_fire_data) :
    fire_branches = fire_event.overlapping_forest_branches
    fire_start_time = fire_event.발생일시
    fire_end_time = fire_event.진화일시
    
    mask = (weather['일시'] >= fire_start_time) & (weather['일시'] <= fire_end_time) & (weather['지점명'].isin(fire_branches))

    weather.loc[mask, '산불'] = 1

    if (index + 1) % 100 == 0:
        print(f"  - {index + 1}/{num_valid_fire_data} 유효 이벤트 처리 완료")

  - 100/665 유효 이벤트 처리 완료
  - 200/665 유효 이벤트 처리 완료
  - 300/665 유효 이벤트 처리 완료
  - 400/665 유효 이벤트 처리 완료
  - 500/665 유효 이벤트 처리 완료
  - 600/665 유효 이벤트 처리 완료


In [ ]:
weather['산불'].sum()

np.int64(7836)

In [ ]:
weather[weather['산불'] == 1]

,지점,지점명,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),실효습도,산불
11949,116,관악(레),2022-05-17 14:00:00,20.4,182.7,2.1,0.0,47.0,23.970000000000006,1
11950,116,관악(레),2022-05-17 15:00:00,22.0,178.4,2.0,0.0,42.0,22.470000000000006,1
11951,116,관악(레),2022-05-17 16:00:00,20.9,207.3,2.5,0.0,47.0,22.920000000000005,1
11952,116,관악(레),2022-05-17 17:00:00,19.8,198.4,2.3,0.0,51.0,25.170000000000005,1
11953,116,관악(레),2022-05-17 18:00:00,18.2,192.1,3.1,0.0,56.0,27.51,1
...,...,...,...,...,...,...,...,...,...,...
14494151,900,두서,2024-08-18 16:00:00,31.2,113.2,1.9,0.0,58.0,29.370000000000005,1
14537296,916,단성,2024-08-31 15:00:00,32.9,357.6,0.9,0.0,44.0,21.810000000000006,1
14537297,916,단성,2024-08-31 16:00:00,33.0,93.3,1.0,0.0,41.0,21.540000000000003,1
14537298,916,단성,2024-08-31 17:00:00,32.8,166.9,1.1,0.0,36.0,19.410000000000004,1


### 산불 데이터와 기상 데이터 매핑


In [ ]:
total_fire_events = len(fire_data)
weather['산불'] = 0 # 칼럼 초기화 (산불이 없는 경우 0)

for index, fire_event in enumerate(fire_data.itertuples()) :
    fire_branches = fire_event.지점명
    fire_start_time = fire_event.발생일시
    fire_end_time = fire_event.진화일시

    mask = (weather['일시'] >= fire_start_time) & (weather['일시'] <= fire_end_time) & (weather['지점명'] == fire_branches)
    weather.loc[mask, '산불'] = 1

    if (index + 1) % 100 == 0:
        print(f"  - {index + 1}/{total_fire_events} 유효 이벤트 처리 완료")
print("\n '산불' 업데이트 완료")

  - 100/1516 유효 이벤트 처리 완료
  - 200/1516 유효 이벤트 처리 완료
  - 300/1516 유효 이벤트 처리 완료
  - 400/1516 유효 이벤트 처리 완료
  - 500/1516 유효 이벤트 처리 완료
  - 600/1516 유효 이벤트 처리 완료
  - 700/1516 유효 이벤트 처리 완료
  - 800/1516 유효 이벤트 처리 완료
  - 900/1516 유효 이벤트 처리 완료
  - 1000/1516 유효 이벤트 처리 완료
  - 1100/1516 유효 이벤트 처리 완료
  - 1200/1516 유효 이벤트 처리 완료
  - 1300/1516 유효 이벤트 처리 완료
  - 1400/1516 유효 이벤트 처리 완료
  - 1500/1516 유효 이벤트 처리 완료

 '산불' 업데이트 완료


In [ ]:
weather

,지점,지점명,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),실효습도,산불
0,12,안면도(감),2022-01-01 00:00:00,-4.5,34.0,2.6,0.0,59.0,17.700000000000003,0
1,12,안면도(감),2022-01-01 01:00:00,-4.6,37.8,2.4,0.0,58.0,29.790000000000003,0
2,12,안면도(감),2022-01-01 02:00:00,-5.0,62.6,2.7,0.0,65.0,31.680000000000003,0
3,12,안면도(감),2022-01-01 03:00:00,-5.0,68.4,2.9,0.0,54.0,29.850000000000005,0
4,12,안면도(감),2022-01-01 04:00:00,-4.9,65.6,2.7,0.0,51.0,26.640000000000004,0
...,...,...,...,...,...,...,...,...,...,...
16418967,996,화동,2024-12-31 19:00:00,-0.5,268.2,3.1,0.0,38.0,18.960000000000004,0
16418968,996,화동,2024-12-31 20:00:00,-0.8,266.5,3.8,0.0,38.0,19.380000000000003,0
16418969,996,화동,2024-12-31 21:00:00,-1.6,262.5,5.2,0.0,46.0,21.78,0
16418970,996,화동,2024-12-31 22:00:00,-2.4,263.6,3.9,0.0,55.0,26.16,0


In [ ]:
weather['산불'].sum()

np.int64(9319)

In [ ]:
weather[weather['산불'] == 1]

,지점,지점명,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),실효습도,산불
11949,116,관악(레),2022-05-17 14:00:00,20.4,182.7,2.1,0.0,47.0,23.970000000000006,1
11950,116,관악(레),2022-05-17 15:00:00,22.0,178.4,2.0,0.0,42.0,22.470000000000006,1
11951,116,관악(레),2022-05-17 16:00:00,20.9,207.3,2.5,0.0,47.0,22.920000000000005,1
11952,116,관악(레),2022-05-17 17:00:00,19.8,198.4,2.3,0.0,51.0,25.170000000000005,1
11953,116,관악(레),2022-05-17 18:00:00,18.2,192.1,3.1,0.0,56.0,27.51,1
...,...,...,...,...,...,...,...,...,...,...
14537298,916,단성,2024-08-31 17:00:00,32.8,166.9,1.1,0.0,36.0,19.410000000000004,1
14537299,916,단성,2024-08-31 18:00:00,30.1,162.6,2.8,0.0,54.0,23.760000000000005,1
14561543,925,생림,2024-09-10 16:00:00,35.4,134.5,2.8,0.0,45.0,22.950000000000003,1
14561544,925,생림,2024-09-10 17:00:00,33.4,122.3,2.5,0.0,51.0,24.750000000000004,1


In [ ]:
weather.to_parquet('weather_fire_label.parquet', index=False)

In [ ]:
9319 / 16418972 * 100 # 산불 발생한 데이터 비율 계산

0.05675751198065262